In [1]:
!pip install pyspark

In [2]:
import pyspark
from pyspark.sql import SparkSession
from google.colab import files

In [3]:
spark = SparkSession.builder.appName('YoutubeDataAnalysis').getOrCreate()

In [4]:
uploaded = files.upload()

Saving comments.csv to comments (1).csv
Saving videos-stats.csv to videos-stats (1).csv


In [6]:
videos_df = spark.read.option("header", True).csv("videos-stats.csv")

In [7]:
videos_df.show(8)

+---+--------------------+-----------+------------+-------+--------+--------+---------+
|_c0|               Title|   Video ID|Published At|Keyword|   Likes|Comments|    Views|
+---+--------------------+-----------+------------+-------+--------+--------+---------+
|  0|Apple Pay Is Kill...|wAZZ-UWGVHI|  2022-08-23|   tech|  3407.0|   672.0| 135612.0|
|  1|The most EXPENSIV...|b3x28s61q3c|  2022-08-24|   tech| 76779.0|  4306.0|1758063.0|
|  2|My New House Gami...|4mgePWWCAmA|  2022-08-23|   tech| 63825.0|  3338.0|1564007.0|
|  3|Petrol Vs Liquid ...|kXiYSI7H2b0|  2022-08-23|   tech| 71566.0|  1426.0| 922918.0|
|  4|Best Back to Scho...|ErMwWXQxHp0|  2022-08-08|   tech| 96513.0|  5155.0|1855644.0|
|  5|Brewmaster Answer...|18fwz9Itbvo|  2021-11-05|   tech| 33570.0|  1643.0| 943119.0|
|  6|Tech Monopolies: ...|jXf04bhcjbg|  2022-06-13|   tech|135047.0|  9367.0|5937790.0|
|  7|I bought the STRA...|2TqOmtTAMRY|  2022-08-07|   tech|216935.0| 12605.0|4782514.0|
+---+--------------------+------

In [8]:
videos_df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Video ID: string (nullable = true)
 |-- Published At: string (nullable = true)
 |-- Keyword: string (nullable = true)
 |-- Likes: string (nullable = true)
 |-- Comments: string (nullable = true)
 |-- Views: string (nullable = true)



In [9]:
videos_df_inf = spark.read.option("header", True).option("inferSchema", True).csv("videos-stats.csv")
videos_df_inf.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Video ID: string (nullable = true)
 |-- Published At: date (nullable = true)
 |-- Keyword: string (nullable = true)
 |-- Likes: double (nullable = true)
 |-- Comments: double (nullable = true)
 |-- Views: double (nullable = true)



In [10]:
videos_df_inf.write.mode("overwrite").option("header", True).parquet("videos-stats-parquet")

In [11]:
videos_parquet_df = spark.read.option("header", True).parquet("videos-stats-parquet")
videos_parquet_df.show(5)

+---+--------------------+-----------+------------+-------+-------+--------+---------+
|_c0|               Title|   Video ID|Published At|Keyword|  Likes|Comments|    Views|
+---+--------------------+-----------+------------+-------+-------+--------+---------+
|  0|Apple Pay Is Kill...|wAZZ-UWGVHI|  2022-08-23|   tech| 3407.0|   672.0| 135612.0|
|  1|The most EXPENSIV...|b3x28s61q3c|  2022-08-24|   tech|76779.0|  4306.0|1758063.0|
|  2|My New House Gami...|4mgePWWCAmA|  2022-08-23|   tech|63825.0|  3338.0|1564007.0|
|  3|Petrol Vs Liquid ...|kXiYSI7H2b0|  2022-08-23|   tech|71566.0|  1426.0| 922918.0|
|  4|Best Back to Scho...|ErMwWXQxHp0|  2022-08-08|   tech|96513.0|  5155.0|1855644.0|
+---+--------------------+-----------+------------+-------+-------+--------+---------+
only showing top 5 rows



In [12]:
videos_parquet_df.createOrReplaceTempView("tb_videos")

In [13]:
spark.sql("SELECT * from tb_videos LIMIT 5").show()

+---+--------------------+-----------+------------+-------+-------+--------+---------+
|_c0|               Title|   Video ID|Published At|Keyword|  Likes|Comments|    Views|
+---+--------------------+-----------+------------+-------+-------+--------+---------+
|  0|Apple Pay Is Kill...|wAZZ-UWGVHI|  2022-08-23|   tech| 3407.0|   672.0| 135612.0|
|  1|The most EXPENSIV...|b3x28s61q3c|  2022-08-24|   tech|76779.0|  4306.0|1758063.0|
|  2|My New House Gami...|4mgePWWCAmA|  2022-08-23|   tech|63825.0|  3338.0|1564007.0|
|  3|Petrol Vs Liquid ...|kXiYSI7H2b0|  2022-08-23|   tech|71566.0|  1426.0| 922918.0|
|  4|Best Back to Scho...|ErMwWXQxHp0|  2022-08-08|   tech|96513.0|  5155.0|1855644.0|
+---+--------------------+-----------+------------+-------+-------+--------+---------+



In [14]:
comments_df = spark.read.option("header",True).option("inferSchema", True).csv("comments.csv")
comments_df.show(5)

+---+-----------+--------------------+-----+---------+
|_c0|   Video ID|             Comment|Likes|Sentiment|
+---+-----------+--------------------+-----+---------+
|  0|wAZZ-UWGVHI|Let's not forget ...| 95.0|      1.0|
|  1|wAZZ-UWGVHI|Here in NZ 50% of...| 19.0|      0.0|
|  2|wAZZ-UWGVHI|I will forever ac...|161.0|      2.0|
|  3|wAZZ-UWGVHI|Whenever I go to ...|  8.0|      0.0|
|  4|wAZZ-UWGVHI|Apple Pay is so c...| 34.0|      2.0|
+---+-----------+--------------------+-----+---------+
only showing top 5 rows



In [15]:
comments_df.write.mode("overwrite").option("header", True).parquet("comments-parquet")